# Extract and display a connectivity matrix from a SONATA circuit

Copyright (c) 2025 Open Brain Institute

Authors: Christoph Pokorny

Last modified: 01.2025

## Summary
This analysis extracts and visualizes a matrix of connection probabilities or #synapses per connection (mean/std/...), grouped by a selected neuron property (layer, m-type, ...).
For details, see the [README](README.md).

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from bluepysnap import Circuit
from connectome_manipulator.connectome_comparison import connectivity
from ipywidgets import widgets, interact

## Circuit selection

A SONATA circuit is assumed to be located under `./analysis_circuit/circuit_config.json`. For using an existing circuit from some other location, modify the `circuit_config = ...` path below accordingly.

In [2]:
# Path to existing circuit config
circuit_config = "./analysis_circuit/circuit_config.json"

assert os.path.exists(circuit_config), f"ERROR: Circuit config '{os.path.split(circuit_config)[1]}' not found!"

Loading SONATA circuit. Selections of the edge population containing the synapses, as well as pre-/post-synaptic node sets defining groups of neurons are possible.

In [3]:
c = Circuit(circuit_config)
e_populations = c.edges.population_names
assert len(e_populations) > 0, "ERROR: No edge population found!"
node_sets = list(c.node_sets.content.keys())
e_popul_wdgt = widgets.Dropdown(options=e_populations, description="Edge population:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
pre_nset_wdgt = widgets.Dropdown(options=[None] + node_sets, description="Pre-synaptic node set:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
post_nset_wdgt = widgets.Dropdown(options=[None] + node_sets, description="Post-synaptic node set:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
display(e_popul_wdgt)
display(pre_nset_wdgt)
display(post_nset_wdgt)

Dropdown(description='Edge population:', layout=Layout(width='max-content'), options=('default',), style=Descr…

Dropdown(description='Pre-synaptic node set:', layout=Layout(width='max-content'), options=(None, 'All', 'Exci…

Dropdown(description='Post-synaptic node set:', layout=Layout(width='max-content'), options=(None, 'All', 'Exc…

## Grouping selection

The connectivity is grouped by one selected (categorical) neuron property, like layer, m-type, etc.

In [4]:
def get_props_categorical(nodes):
    is_cat = nodes.get([]).dtypes.apply(lambda _x: isinstance(_x, pd.CategoricalDtype))
    props = is_cat[is_cat].index.to_list()  # Select categorical properties
    for _p in ["region", "layer", "mtype", "etype", "synapse_class"]:
        if _p in nodes.property_names:
            props = props + [_p]  # Add certain properties, even if not categorical
    return np.unique(props)

def get_props_float(nodes):
    props = nodes.property_dtypes[nodes.property_dtypes == float]
    return props.index.values

node_props = np.intersect1d(get_props_categorical(c.edges[e_popul_wdgt.value].source),
                            get_props_categorical(c.edges[e_popul_wdgt.value].target))
float_props = np.intersect1d(get_props_float(c.edges[e_popul_wdgt.value].source),
                            get_props_float(c.edges[e_popul_wdgt.value].target))
selected_props = list(np.intersect1d(["x", "y", "z", "meh"], float_props))

groupby_wdgt = widgets.Dropdown(options=node_props, value="layer" if "layer" in node_props else node_props[0], description="Group-by:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
distance_wdgt = widgets.IntSlider(value=100, min=10, max=1000, step=10, description="Max distance", readout=True)
dist_props_wdgt = widgets.SelectMultiple(rows=10, options=float_props, value=selected_props, description="Distance properties")
use_dist_wdgt = widgets.Checkbox(value=False, description="Use distance cutoff")

def display_fcn(use_dist_val):
    if use_dist_val:
        #display(distance_wdgt)
        distance_wdgt.layout.visibility = "visible"
        dist_props_wdgt.layout.visibility = "visible"
        #display(dist_props_wdgt)
    else:
        distance_wdgt.layout.visibility = "hidden"
        dist_props_wdgt.layout.visibility = "hidden"
i = widgets.interactive(display_fcn, use_dist_val=use_dist_wdgt)
display(groupby_wdgt)
display(i)
display(distance_wdgt)
display(dist_props_wdgt)


Dropdown(description='Group-by:', index=1, layout=Layout(width='max-content'), options=(np.str_('etype'), np.s…

interactive(children=(Checkbox(value=False, description='Use distance cutoff'), Output()), _dom_classes=('widg…

IntSlider(value=100, description='Max distance', layout=Layout(visibility='hidden'), max=1000, min=10, step=10…

SelectMultiple(description='Distance properties', index=(8, 9, 10), layout=Layout(visibility='hidden'), option…

## Connectivity extraction

Extract the connectivity for all pairs of groups using functionality from [connectome-manipulator](https://github.com/openbraininstitute/connectome-manipulator).

In [5]:
max_dist_val, props_val = None, None
if use_dist_wdgt.value:
    max_dist_val = distance_wdgt.value
    props_val = list(dist_props_wdgt.value)
conn_dict = connectivity.compute(c, sel_src={"node_set": pre_nset_wdgt.value}, sel_dest={"node_set": post_nset_wdgt.value}, edges_popul_name=e_popul_wdgt.value, group_by=groupby_wdgt.value,
                                 max_distance=max_dist_val, props_for_distance=props_val)

INFO: Computing connectivity (group_by=layer, sel_src={'node_set': None}, sel_dest={'node_set': None}, N=6x6 groups)


100% |########################################################################|


## Interactive visualization

Interactive visualization of the connectivity matrix. The user can select what characteristic to display, such as connection probabilities or mean/max/min/SEM/std of #synapses per connection (SEM ... standard error of the mean). Also, empty groups (i.e., groups w/o any neurons or connections) can be excluded using a checkbox.

In [6]:
# Interactive plot function
def plot_fct(res_sel, empty_sel):
    def filter_conn_dict(conn_dict, res_sel):
        out_dict = {}
        p = conn_dict["conn_prob"]["data"]
        pre_sel = np.any(p > 0.0, axis=1)
        post_sel = np.any(p > 0.0, axis=0)
        out_dict[res_sel] = conn_dict[res_sel].copy()
        out_dict[res_sel]["data"] = out_dict[res_sel]["data"][pre_sel, :][:, post_sel]
        out_dict["common"] = conn_dict["common"].copy()
        out_dict["common"]["src_group_values"] = [_v for _v, _s in zip(out_dict["common"]["src_group_values"], pre_sel) if _s]
        out_dict["common"]["tgt_group_values"] = [_v for _v, _s in zip(out_dict["common"]["tgt_group_values"], post_sel) if _s]
        return out_dict
    
    if empty_sel:
        plot_dict = conn_dict
    else:
        plot_dict = filter_conn_dict(conn_dict, res_sel)

    if plot_dict[res_sel]["data"].size > 0:
        vmax = np.max(plot_dict[res_sel]["data"])
        if vmax == 0.0:
            vmax = 1.0
        connectivity.plot(plot_dict[res_sel], plot_dict["common"], vmin=0.0, vmax=vmax, group_by=groupby_wdgt.value)
    else:
        plt.figure()
        plt.text(0, 0, "Nothing to show...")
        plt.axis("off")
        plt.axis("tight")
        plt.show()

In [7]:
res_sel_opt = [(_v["unit"], _k) for _k, _v in sorted(conn_dict.items()) if "unit" in _v]
res_sel_wdgt = widgets.Dropdown(options=res_sel_opt, description="Display:", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
empty_sel_wdgt = widgets.Checkbox(value=True, description="Show empty groups", style={"description_width": "auto"}, layout=widgets.Layout(width="max-content"))
iplot = interact(plot_fct, res_sel=res_sel_wdgt, empty_sel=empty_sel_wdgt)

interactive(children=(Dropdown(description='Display:', layout=Layout(width='max-content'), options=(('Conn. pr…